In [ ]:
"""
Created on: Mon. 29 Aug. 2022
Author: Mélina Verger
"""

# For data manipulation
import pandas as pd

# To handle ZIP files
import zipfile


## Load data sets

In [ ]:
zf = zipfile.ZipFile("./data/data.zip") 

In [ ]:
studentInfo = pd.read_csv(zf.open("studentInfo.csv"))
studentAssessment = pd.read_csv(zf.open("studentAssessment.csv"))
studentVle = pd.read_csv(zf.open("studentVle.csv"))
assessments = pd.read_csv(zf.open("assessments.csv"))

## Merge assessments information

In [ ]:
studentAssessment

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0
...,...,...,...,...,...
173907,37443,527538,227,0,60.0
173908,37443,534672,229,0,100.0
173909,37443,546286,215,0,80.0
173910,37443,546724,230,0,100.0


In [ ]:
assessments

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0
...,...,...,...,...,...,...
201,GGG,2014J,37443,CMA,229.0,0.0
202,GGG,2014J,37435,TMA,61.0,0.0
203,GGG,2014J,37436,TMA,124.0,0.0
204,GGG,2014J,37437,TMA,173.0,0.0


In [ ]:
studentScore = pd.merge(studentAssessment, assessments, how="inner", on="id_assessment")
studentScore = studentScore.drop(columns=["date_submitted", "is_banked", "date"])

In [ ]:
studentScore

,id_assessment,id_student,score,code_module,code_presentation,assessment_type,weight
0,1752,11391,78.0,AAA,2013J,TMA,10.0
1,1752,28400,70.0,AAA,2013J,TMA,10.0
2,1752,31604,72.0,AAA,2013J,TMA,10.0
3,1752,32885,69.0,AAA,2013J,TMA,10.0
4,1752,38053,79.0,AAA,2013J,TMA,10.0
...,...,...,...,...,...,...,...
173907,37443,527538,60.0,GGG,2014J,CMA,0.0
173908,37443,534672,100.0,GGG,2014J,CMA,0.0
173909,37443,546286,80.0,GGG,2014J,CMA,0.0
173910,37443,546724,100.0,GGG,2014J,CMA,0.0


## Aggregate assessments information

In [ ]:
code_module = studentScore["code_module"].unique()
code_presentation = studentScore["code_presentation"].unique()
id_student = studentScore["id_student"].unique()

In [ ]:
studentScore["weight"].replace(0, 1, inplace=True)  # to avoid zero division

In [ ]:
# Create empty dataframe
new_df = pd.DataFrame(columns=["id_student", "code_module", "code_presentation", "weighted_score"])

In [174]:
i = 0
for id_stu in id_student:
    for code_mod in code_module:
        for code_pres in code_presentation:
            mini_df = studentScore.loc[(studentScore["id_student"] == id_stu) & (studentScore["code_module"] == code_mod) & (studentScore["code_presentation"] == code_pres)]
            if len(mini_df) > 0: # if data exists
                i +=1
                sum_weight = mini_df["weight"].sum()
                numerator = (mini_df["score"] * mini_df["weight"]).sum()
                weighted_score = numerator/sum_weight
                new_line = pd.DataFrame([(id_stu, code_mod, code_pres, weighted_score)], columns=["id_student", "code_module", "code_presentation", "weighted_score"])
                new_df = pd.concat([new_df, new_line], ignore_index=True)
                print("Iteration {} / 173912".format(i), end="\r", flush=True)  # overwrite
                # ~ 206 rows per minute => 844' for 173,912 rows => 14h
                # 146' in the end for 25843 / 173912 iterations


In [175]:
new_df.to_csv("./data/new_df.csv", index=False)

In [ ]:
studentScore.groupby(["code_module", "code_presentation", "id_student"]).sum()

id_assessment  score  weight
code_module code_presentation id_student                              
AAA         2013J             11391                8770  410.0   100.0
                              28400                8770  332.0   100.0
                              31604                8770  380.0   100.0
                              32885                8770  272.0   100.0
                              38053                8770  340.0   100.0
...                                                 ...    ...     ...
GGG         2014J             2620947            336951  800.0     0.0
                              2645731            336951  793.0     0.0
                              2648187            336951  690.0     0.0
                              2679821             74873  183.0     0.0
                              2684003            262066  580.0     0.0

[25843 rows x 3 columns]

## Aggregate click information

In [ ]:
studentVle

,code_module,code_presentation,id_student,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1
...,...,...,...,...,...,...
10655275,GGG,2014J,675811,896943,269,3
10655276,GGG,2014J,675578,896943,269,1
10655277,GGG,2014J,654064,896943,269,3
10655278,GGG,2014J,654064,896939,269,1


In [ ]:
studentClick = studentVle.groupby(["code_module", "code_presentation", "id_student"]).sum()
studentClick = studentClick.drop(columns=["id_site", "date"])

In [ ]:
studentClick

sum_click
code_module code_presentation id_student           
AAA         2013J             11391             934
                              28400            1435
                              30268             281
                              31604            2158
                              32885            1034
...                                             ...
GGG         2014J             2640965            41
                              2645731           893
                              2648187           312
                              2679821           275
                              2684003           616

[29228 rows x 1 columns]

In [ ]:
# To check the group by (change the id_student number and verify the sum)
# studentVle[studentVle["code_module"]=="AAA"][studentVle["code_presentation"]=="2013J"][studentVle["id_student"]==30268]["sum_click"].sum()

## Merge studentInfo with click

In [ ]:
studentAll = pd.merge(studentInfo, studentClick, how="inner", on=["code_module", "code_presentation", "id_student"])
# It is normal if with inner join the number of rows goes from 32593 (rows from studentInfo) to 29228 (rows from studentClick) because of the duplicates in studentInfo

In [ ]:
pd.merge(studentAll, studentClick, how="inner", on=["code_module", "code_presentation", "id_student"])


,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,sum_click_x,sum_click_y
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,934,934
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,1435,1435
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,281,281
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,2158,2158
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,1034,1034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29223,GGG,2014J,2640965,F,Wales,Lower Than A Level,10-20,0-35,0,30,N,Fail,41,41
29224,GGG,2014J,2645731,F,East Anglian Region,Lower Than A Level,40-50%,35-55,0,30,N,Distinction,893,893
29225,GGG,2014J,2648187,F,South Region,A Level or Equivalent,20-30%,0-35,0,30,Y,Pass,312,312
29226,GGG,2014J,2679821,F,South East Region,Lower Than A Level,90-100%,35-55,0,30,N,Withdrawn,275,275
